In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.utils.data as Data
from torch.autograd import Variable

from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv("../data/raw_data.csv")

In [3]:
df_train, df_test = train_test_split(df, test_size = 0.2)

In [4]:
df_train.to_csv("train.csv")
df_test.to_csv("test.csv")

In [5]:
df_train = pd.read_csv("train.csv")

In [15]:
df_train.shape

(20440, 30)

In [6]:
model_file_path = "model.pkl"
params = ["grna_target_sequence", "target_sequence"]

In [7]:
# Drop NA
params.append("cleavage_freq")
for col in params:
    df_train = df_train[df_train[col].notna()]
params.remove("cleavage_freq")

### Get X

In [8]:
# Select
df_X = df_train[params]

# Convert to string
df_X = df_X[params].convert_dtypes()

#Remove dash
for col in df_X.select_dtypes(exclude = ["number"]).columns:
    df_X[col] = [
        seq.replace("-", "")
        for seq in df_X[col]
    ]

# Pad
for col in df_X.select_dtypes(exclude = ["number"]).columns:
    df_X[col] = df_X[col].str.pad(width = 50, side = "right", fillchar = "X")

# Encode
def encode_nt(nt:str) -> int:
    assert len(nt) == 1
    encoding_dict = {
        'X': 0,
        'A': 0.25,
        'T': 0.5,
        'G': 0.75,
        'C': 1
    }
    return encoding_dict.get(nt.upper())
def encode_seq(seq:str):
    encoding = [
        encode_nt(nt)
        for nt in seq
    ]
    return np.array(encoding)
def encode(df):
    df["grna_target_sequence"] = [
        encode_seq(seq)
        for seq in df["grna_target_sequence"]
    ]
    df["target_sequence"] = [
        encode_seq(seq)
        for seq in df["target_sequence"]
    ]
    return df
df_X = encode(df_X)

# fold
df_X["stacked"] = df_X["grna_target_sequence"].apply(lambda x: x.tolist()) + df_X["target_sequence"].apply(lambda x: x.tolist())
df_X["stacked"] = df_X["stacked"].apply(lambda x: np.array(x))

# Tensorfy
temp = []
for i in df_X["stacked"]:
    temp.append(i)
X = torch.from_numpy(np.array(temp).astype(np.float32))

In [9]:
X.dtype

torch.float32

### Get y

In [10]:
df_y = df_train["cleavage_freq"]
y = torch.Tensor(np.array(df_y).reshape(df_y.shape[0], 1).astype(np.float32))

In [11]:
y

tensor([[2.4910e-03],
        [1.7341e-02],
        [6.7180e-04],
        ...,
        [1.7294e-04],
        [7.9770e-06],
        [3.0000e-02]])

In [12]:
y.dtype

torch.float32

### Train

In [13]:
input_dim = X.shape[1]
X = Variable(X)
y = Variable(y)
model = nn.Linear(input_dim, 1)
loss_func = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 1e-5)
BATCH_SIZE = 5
EPOCH = 5
torch_dataset = Data.TensorDataset(X, y)
loader = Data.DataLoader(
    dataset = torch_dataset,
    batch_size = BATCH_SIZE,
    shuffle = True,
    num_workers = 2
)
for epoch in range(EPOCH):
    for step, (batch_x, batch_y) in enumerate(loader):
        prediction = model(batch_x)
        print(prediction)
        loss = loss_func(prediction, batch_y)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

tensor([[0.0425],
        [0.2758],
        [0.2845],
        [0.0659],
        [0.0664]], grad_fn=<AddmmBackward>)
tensor([[0.2949],
        [0.2249],
        [0.1881],
        [0.0132],
        [0.0463]], grad_fn=<AddmmBackward>)
tensor([[0.3946],
        [0.3009],
        [0.1274],
        [0.1412],
        [0.3154]], grad_fn=<AddmmBackward>)
tensor([[0.4255],
        [0.2819],
        [0.1453],
        [0.0629],
        [0.0250]], grad_fn=<AddmmBackward>)
tensor([[0.0421],
        [0.3249],
        [0.2092],
        [0.2631],
        [0.3554]], grad_fn=<AddmmBackward>)
tensor([[0.2983],
        [0.3294],
        [0.1545],
        [0.1372],
        [0.3636]], grad_fn=<AddmmBackward>)
tensor([[0.2511],
        [0.2765],
        [0.2697],
        [0.2934],
        [0.1990]], grad_fn=<AddmmBackward>)
tensor([[0.1542],
        [0.1152],
        [0.1701],
        [0.2427],
        [0.2047]], grad_fn=<AddmmBackward>)
tensor([[0.4214],
        [0.3149],
        [0.0500],
        [0.1806],


tensor([[0.3686],
        [0.2899],
        [0.1999],
        [0.3588],
        [0.2572]], grad_fn=<AddmmBackward>)
tensor([[0.0551],
        [0.4697],
        [0.1957],
        [0.4200],
        [0.3531]], grad_fn=<AddmmBackward>)
tensor([[ 0.3982],
        [ 0.2329],
        [-0.0479],
        [ 0.2585],
        [ 0.1663]], grad_fn=<AddmmBackward>)
tensor([[0.1973],
        [0.1266],
        [0.1167],
        [0.2008],
        [0.2804]], grad_fn=<AddmmBackward>)
tensor([[0.1699],
        [0.0021],
        [0.0449],
        [0.4184],
        [0.0215]], grad_fn=<AddmmBackward>)
tensor([[0.2721],
        [0.2959],
        [0.1440],
        [0.0875],
        [0.1709]], grad_fn=<AddmmBackward>)
tensor([[0.2681],
        [0.1329],
        [0.0925],
        [0.2176],
        [0.1534]], grad_fn=<AddmmBackward>)
tensor([[0.2125],
        [0.1947],
        [0.0992],
        [0.1427],
        [0.1971]], grad_fn=<AddmmBackward>)
tensor([[0.3308],
        [0.3897],
        [0.1478],
        [0.04

tensor([[0.4484],
        [0.0347],
        [0.2111],
        [0.2634],
        [0.1211]], grad_fn=<AddmmBackward>)
tensor([[0.1262],
        [0.0427],
        [0.1414],
        [0.2238],
        [0.1200]], grad_fn=<AddmmBackward>)
tensor([[0.1546],
        [0.1425],
        [0.1944],
        [0.1617],
        [0.1312]], grad_fn=<AddmmBackward>)
tensor([[ 0.0676],
        [-0.0635],
        [ 0.1294],
        [ 0.0978],
        [ 0.3076]], grad_fn=<AddmmBackward>)
tensor([[0.0712],
        [0.1910],
        [0.2420],
        [0.1342],
        [0.0107]], grad_fn=<AddmmBackward>)
tensor([[ 0.0808],
        [ 0.2346],
        [-0.0204],
        [ 0.3603],
        [-0.0069]], grad_fn=<AddmmBackward>)
tensor([[0.1228],
        [0.0496],
        [0.0185],
        [0.1462],
        [0.0288]], grad_fn=<AddmmBackward>)
tensor([[0.0187],
        [0.0323],
        [0.0842],
        [0.1105],
        [0.0699]], grad_fn=<AddmmBackward>)
tensor([[0.2426],
        [0.0768],
        [0.1649],
        

tensor([[0.4206],
        [0.1121],
        [0.2643],
        [0.2556],
        [0.0982]], grad_fn=<AddmmBackward>)
tensor([[0.1752],
        [0.1399],
        [0.2348],
        [0.0200],
        [0.2293]], grad_fn=<AddmmBackward>)
tensor([[0.2319],
        [0.0222],
        [0.2752],
        [0.0674],
        [0.1517]], grad_fn=<AddmmBackward>)
tensor([[0.3189],
        [0.2632],
        [0.1175],
        [0.2476],
        [0.0841]], grad_fn=<AddmmBackward>)
tensor([[ 0.2446],
        [-0.0117],
        [-0.0410],
        [ 0.1750],
        [ 0.2716]], grad_fn=<AddmmBackward>)
tensor([[ 0.0821],
        [ 0.2265],
        [ 0.0575],
        [ 0.0526],
        [-0.0222]], grad_fn=<AddmmBackward>)
tensor([[0.1914],
        [0.0980],
        [0.2614],
        [0.2117],
        [0.2351]], grad_fn=<AddmmBackward>)
tensor([[0.0006],
        [0.0384],
        [0.1173],
        [0.3967],
        [0.0998]], grad_fn=<AddmmBackward>)
tensor([[0.3007],
        [0.0945],
        [0.2029],
        

tensor([[0.0866],
        [0.2373],
        [0.0330],
        [0.3912],
        [0.2610]], grad_fn=<AddmmBackward>)
tensor([[0.1827],
        [0.1347],
        [0.3569],
        [0.1017],
        [0.0158]], grad_fn=<AddmmBackward>)
tensor([[0.2061],
        [0.2173],
        [0.2024],
        [0.1791],
        [0.2580]], grad_fn=<AddmmBackward>)
tensor([[-0.0037],
        [ 0.2248],
        [ 0.2463],
        [ 0.2340],
        [ 0.0194]], grad_fn=<AddmmBackward>)
tensor([[ 0.2781],
        [ 0.2785],
        [-0.0907],
        [ 0.1447],
        [ 0.2581]], grad_fn=<AddmmBackward>)
tensor([[0.3189],
        [0.1131],
        [0.1564],
        [0.2613],
        [0.0996]], grad_fn=<AddmmBackward>)
tensor([[0.0046],
        [0.0308],
        [0.0823],
        [0.1050],
        [0.2033]], grad_fn=<AddmmBackward>)
tensor([[0.0476],
        [0.2329],
        [0.1212],
        [0.2238],
        [0.2301]], grad_fn=<AddmmBackward>)
tensor([[0.1031],
        [0.0773],
        [0.3299],
        

...

tensor([[ 0.1795],
        [-0.0903],
        [-0.0590],
        [ 0.1173],
        [ 0.2017]], grad_fn=<AddmmBackward>)
tensor([[ 0.0481],
        [ 0.1101],
        [-0.0948],
        [ 0.0728],
        [ 0.0279]], grad_fn=<AddmmBackward>)
tensor([[ 0.0362],
        [ 0.1766],
        [-0.0199],
        [ 0.1132],
        [ 0.0252]], grad_fn=<AddmmBackward>)
tensor([[-0.0253],
        [-0.0420],
        [ 0.0120],
        [ 0.1345],
        [-0.0496]], grad_fn=<AddmmBackward>)
tensor([[ 0.0039],
        [ 0.0759],
        [-0.0373],
        [ 0.0344],
        [ 0.1041]], grad_fn=<AddmmBackward>)
tensor([[0.0718],
        [0.0435],
        [0.1187],
        [0.0276],
        [0.0715]], grad_fn=<AddmmBackward>)
tensor([[ 0.1625],
        [ 0.0463],
        [-0.0370],
        [-0.0803],
        [ 0.0534]], grad_fn=<AddmmBackward>)
tensor([[ 0.1064],
        [ 0.0054],
        [-0.1431],
        [ 0.0776],
        [-0.1579]], grad_fn=<AddmmBackward>)
tensor([[ 0.1458],
        [ 0.1103],

tensor([[ 0.0197],
        [-0.0298],
        [ 0.0131],
        [-0.1116],
        [ 0.0040]], grad_fn=<AddmmBackward>)
tensor([[-0.1704],
        [ 0.0057],
        [-0.0077],
        [ 0.1645],
        [ 0.0398]], grad_fn=<AddmmBackward>)
tensor([[ 0.1585],
        [-0.1093],
        [ 0.1913],
        [-0.0531],
        [ 0.0603]], grad_fn=<AddmmBackward>)
tensor([[-0.1176],
        [-0.0643],
        [ 0.0477],
        [ 0.0056],
        [-0.0298]], grad_fn=<AddmmBackward>)
tensor([[ 0.0768],
        [-0.0854],
        [-0.0256],
        [ 0.0522],
        [ 0.0735]], grad_fn=<AddmmBackward>)
tensor([[ 0.0338],
        [ 0.0879],
        [ 0.0133],
        [-0.0292],
        [-0.0311]], grad_fn=<AddmmBackward>)
tensor([[ 0.0669],
        [-0.0058],
        [ 0.0183],
        [ 0.0816],
        [ 0.1410]], grad_fn=<AddmmBackward>)
tensor([[-0.0364],
        [ 0.2282],
        [-0.0470],
        [ 0.0870],
        [ 0.0872]], grad_fn=<AddmmBackward>)
tensor([[ 0.1206],
        [ 0.0

tensor([[0.0390],
        [0.2132],
        [0.0012],
        [0.0568],
        [0.0186]], grad_fn=<AddmmBackward>)
tensor([[ 0.1206],
        [ 0.0770],
        [-0.0324],
        [-0.0555],
        [-0.0080]], grad_fn=<AddmmBackward>)
tensor([[ 0.0175],
        [ 0.0100],
        [-0.0325],
        [-0.0326],
        [ 0.0930]], grad_fn=<AddmmBackward>)
tensor([[ 0.1417],
        [ 0.0968],
        [-0.1351],
        [-0.0120],
        [-0.0145]], grad_fn=<AddmmBackward>)
tensor([[ 0.0405],
        [ 0.0485],
        [-0.0117],
        [-0.0304],
        [ 0.0831]], grad_fn=<AddmmBackward>)
tensor([[ 0.0089],
        [-0.0376],
        [ 0.1168],
        [-0.0017],
        [-0.0410]], grad_fn=<AddmmBackward>)
tensor([[ 0.0141],
        [ 0.1791],
        [ 0.1480],
        [-0.0778],
        [-0.0155]], grad_fn=<AddmmBackward>)
tensor([[-0.1119],
        [ 0.0461],
        [ 0.1121],
        [ 0.0469],
        [ 0.0322]], grad_fn=<AddmmBackward>)
tensor([[-0.1294],
        [ 0.1887],

tensor([[ 0.1191],
        [-0.0186],
        [ 0.1341],
        [-0.1016],
        [ 0.0688]], grad_fn=<AddmmBackward>)
tensor([[ 8.2374e-02],
        [-6.4785e-02],
        [-4.8279e-02],
        [ 1.3972e-02],
        [-6.9417e-05]], grad_fn=<AddmmBackward>)
tensor([[ 0.0606],
        [-0.0890],
        [-0.0857],
        [ 0.0441],
        [ 0.1377]], grad_fn=<AddmmBackward>)
tensor([[-0.0616],
        [ 0.1255],
        [ 0.0124],
        [ 0.0064],
        [ 0.1176]], grad_fn=<AddmmBackward>)
tensor([[-0.0401],
        [-0.1147],
        [-0.0431],
        [ 0.1182],
        [ 0.1494]], grad_fn=<AddmmBackward>)
tensor([[-0.0374],
        [ 0.0904],
        [-0.1172],
        [ 0.0327],
        [ 0.2086]], grad_fn=<AddmmBackward>)
tensor([[-0.0354],
        [-0.1301],
        [-0.0939],
        [ 0.0831],
        [-0.0211]], grad_fn=<AddmmBackward>)
tensor([[ 0.0008],
        [-0.0600],
        [ 0.2336],
        [ 0.1992],
        [ 0.1046]], grad_fn=<AddmmBackward>)
tensor([[ 0.

tensor([[ 0.0742],
        [-0.1083],
        [ 0.1792],
        [ 0.1559],
        [ 0.0126]], grad_fn=<AddmmBackward>)
tensor([[0.0011],
        [0.0765],
        [0.0589],
        [0.0086],
        [0.0938]], grad_fn=<AddmmBackward>)
tensor([[ 0.0111],
        [-0.0673],
        [ 0.0071],
        [ 0.0149],
        [-0.0950]], grad_fn=<AddmmBackward>)
tensor([[ 0.0189],
        [ 0.0396],
        [ 0.0006],
        [ 0.1716],
        [-0.0908]], grad_fn=<AddmmBackward>)
tensor([[-0.0861],
        [-0.0527],
        [-0.0046],
        [ 0.1160],
        [-0.1643]], grad_fn=<AddmmBackward>)
tensor([[ 0.0594],
        [-0.1881],
        [ 0.2347],
        [ 0.0167],
        [ 0.0117]], grad_fn=<AddmmBackward>)
tensor([[ 0.0502],
        [-0.0644],
        [-0.0245],
        [ 0.1410],
        [ 0.2039]], grad_fn=<AddmmBackward>)
tensor([[-0.0138],
        [-0.1228],
        [-0.0292],
        [ 0.0721],
        [ 0.2149]], grad_fn=<AddmmBackward>)
tensor([[ 0.0504],
        [ 0.1192],